In [318]:
import pandas as pd
import pickle
from scipy.stats import poisson


grupos=pickle.load(open("Grupos2026","rb"))
datos_mundial=pd.read_csv("Mundiales limpio.csv")
fixtures=pd.read_csv("fixtures.csv")

In [319]:
datos_mundial

,Local,Visitante,Año,Goles Local,Goles Visitante,Goles Totales
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
923,England,France,2022,1,2,3
924,Argentina,Croatia,2022,3,0,3
925,France,Morocco,2022,2,0,2
926,Croatia,Morocco,2022,2,1,3


In [320]:
df_local=datos_mundial[["Local","Goles Local","Goles Visitante"]]
df_visitante=datos_mundial[["Visitante","Goles Local","Goles Visitante"]]
df_visitante

,Visitante,Goles Local,Goles Visitante
0,Mexico,4,1
1,France,1,0
2,Mexico,3,0
3,France,1,0
4,Mexico,6,3
...,...,...,...
923,France,1,2
924,Croatia,3,0
925,Morocco,2,0
926,Morocco,2,1


In [321]:
df_local=df_local.rename(columns={"Local":"Equipo","Goles Local":"Goles Anotados","Goles Visitante":"Goles Recibidos"})
df_visitante=df_visitante.rename(columns={"Visitante":"Equipo","Goles Local":"Goles Recibidos","Goles Visitante":"Goles Anotados"})
df_visitante

,Equipo,Goles Recibidos,Goles Anotados
0,Mexico,4,1
1,France,1,0
2,Mexico,3,0
3,France,1,0
4,Mexico,6,3
...,...,...,...
923,France,1,2
924,Croatia,3,0
925,Morocco,2,0
926,Morocco,2,1


In [322]:
df_equipos_fuerza=pd.concat([df_local,df_visitante],ignore_index=True).groupby("Equipo").mean()
df_equipos_fuerza

,Goles Anotados,Goles Recibidos
Equipo,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.752941,1.164706
Australia,0.850000,1.850000
Austria,1.576923,1.692308
...,...,...
Uruguay,1.553571,1.303571
Wales,0.625000,1.250000
West Germany,2.050847,1.254237


In [323]:
def predecir_puntos (local,visitante):
    if local in df_equipos_fuerza.index and visitante in df_equipos_fuerza.index:
        lamb_local=df_equipos_fuerza.at[local,"Goles Anotados"]*df_equipos_fuerza.at[visitante,"Goles Recibidos"]
        lamb_visitante=df_equipos_fuerza.at[visitante,"Goles Anotados"]*df_equipos_fuerza.at[local,"Goles Recibidos"]
        prob_local,prob_visitante,prob_empate=0,0,0
        for x in range(0,11):
            for y in range(0,11):
                p=poisson.pmf(x,lamb_local)*poisson.pmf(y,lamb_visitante)
                if x==y:
                    prob_empate+=p
                elif x>y:
                    prob_local+=p
                else:
                    prob_visitante+=p
        puntos_local=3*prob_local+prob_empate
        puntos_visitante=3*prob_visitante+prob_empate
        return(puntos_local,puntos_visitante)
    else:
        return(0,0)
        
        


In [324]:
predecir_puntos("Mexico","Argentina")



(np.float64(0.5221698773843116), np.float64(2.331588433187245))

In [325]:
df_fixture_grupo_48 = fixtures[:72].copy()
df_fixture_ronda_32 = fixtures[72:88].copy()
df_fixture_ronda_16 = fixtures[88:96].copy()
df_fixture_cuartos = fixtures[96:100].copy()
df_fixture_semi = fixtures[100:102].copy()
df_fixture_final = fixtures[102:].copy()


# Limpieza de nombres "sucios" del repechaje
# (Estos son ejemplos, revisa cuáles tienes tú en tu DataFrame)

reemplazos = {
    'UEFA Path A winner': 'Wales',
    'UEFA Path A winner[a]': 'Wales',# O el que tú creas que ganará
    'UEFA Path B winner': 'Ukraine',
    'UEFA Path B winner[a]': 'Ukraine',
    'UEFA Path C winner': 'Georgia',
    'UEFA Path C winner[a]': 'Georgia',
    'UEFA Path D winner': 'Iceland',
    'UEFA Path D winner[a]': 'Iceland',# Quítale también el corchete [a]
    'AFC v CONMEBOL winner': 'Peru',
    'IC Play-off 1 Winner': 'DR Congo',      # O 'Jamaica'
    'IC Path 1 winner': 'DR Congo',
    'IC Path 1 winner[a]': 'DR Congo',
    "Mexico (H)":"Mexico",
    "Canada (H)":"Canada",
    "United States (H)":"United States",
    # Repechaje Intercontinental 2 (Asia/Conmebol/Concacaf)
    'IC Play-off 2 Winner': 'Iraq',          # O 'Bolivia'
    'IC Path 2 winner': 'Iraq',
    'IC Path 2 winner[a]': 'Iraq'
    # Agrega todos los que veas raros
}
for grupo in grupos:
    # 'replace' busca el texto sucio y pone el limpio
    grupos[grupo]['Equipo'] = grupos[grupo]['Equipo'].replace(reemplazos)
    
    # Extra: Quitamos espacios vacíos al inicio o final por si acaso
    grupos[grupo]['Equipo'] = grupos[grupo]['Equipo'].str.strip()

# 1. Reemplazamos en la columna 'home' (Local)
df_fixture_grupo_48['Local'] = df_fixture_grupo_48['Local'].replace(reemplazos)

# 2. Reemplazamos en la columna 'away' (Visitante)
df_fixture_grupo_48['Visitante'] = df_fixture_grupo_48['Visitante'].replace(reemplazos)

# 3. Y MUY IMPORTANTE: Limpia los espacios vacíos que a veces quedan
df_fixture_grupo_48['Local'] = df_fixture_grupo_48['Local'].str.strip()
df_fixture_grupo_48['Visitante'] = df_fixture_grupo_48['Visitante'].str.strip()

In [344]:
for grupo in grupos:
    equipo_en_grupos=grupos[grupo]["Equipo"].values
    df_fixture_group_6 = df_fixture_grupo_48[df_fixture_grupo_48['Local'].isin(equipo_en_grupos)]
    for index, row in df_fixture_group_6.iterrows():
        local,visitante=row["Local"],row["Visitante"]
        puntos_local,puntos_visitante=predecir_puntos(local,visitante)
        grupos[grupo].loc[grupos[grupo]["Equipo"]==local, "Pts"]+=puntos_local
        grupos[grupo].loc[grupos[grupo]["Equipo"]==visitante, "Pts"]+=puntos_visitante
    
    grupos[grupo]=grupos[grupo].sort_values("Pts", ascending=False).reset_index()
    grupos[grupo]=grupos[grupo][["Equipo","Pts"]]
    grupos[grupo]=grupos[grupo].round(0)
grupos       

{'Grupo A':          Equipo   Pts
 0  South Africa  10.0
 1        Mexico  10.0
 2   South Korea   8.0
 3       Iceland   6.0,
 'Grupo B':         Equipo   Pts
 0  Switzerland  14.0
 1        Wales  10.0
 2       Canada   4.0
 3        Qatar   4.0,
 'Grupo C':      Equipo   Pts
 0    Brazil  14.0
 1   Morocco  10.0
 2  Scotland   8.0
 3     Haiti   0.0,
 'Grupo D':           Equipo  Pts
 0       Paraguay  8.0
 1  United States  6.0
 2      Australia  4.0
 3        Georgia  0.0,
 'Grupo E':         Equipo  Pts
 0      Germany  8.0
 1      Ecuador  4.0
 2  Ivory Coast  4.0
 3      Curaçao  0.0,
 'Grupo F':         Equipo   Pts
 0  Netherlands  14.0
 1        Japan   8.0
 2      Ukraine   6.0
 3      Tunisia   6.0,
 'Grupo G':         Equipo   Pts
 0      Belgium  14.0
 1         Iran   8.0
 2        Egypt   8.0
 3  New Zealand   4.0,
 'Grupo H':          Equipo  Pts
 0         Spain  8.0
 1       Uruguay  8.0
 2  Saudi Arabia  0.0
 3    Cape Verde  0.0,
 'Grupo I':     Equipo   Pts
 0   

In [327]:
df_fixture_ronda_32

,Local,Marcador,Visitante,Año
72,Runner-up Group A,Match 73,Runner-up Group B,2026
73,Winner Group C,Match 76,Runner-up Group F,2026
74,Winner Group E,Match 74,3rd Group A/B/C/D/F,2026
75,Winner Group F,Match 75,Runner-up Group C,2026
76,Runner-up Group E,Match 78,Runner-up Group I,2026
77,Winner Group I,Match 77,3rd Group C/D/F/G/H,2026
78,Winner Group A,Match 79,3rd Group C/E/F/H/I,2026
79,Winner Group L,Match 80,3rd Group E/H/I/J/K,2026
80,Winner Group G,Match 82,3rd Group A/E/H/I/J,2026
81,Winner Group D,Match 81,3rd Group B/E/F/I/J,2026


In [343]:

clasificados = {}
terceros_lugares = []

for grupo, tabla in grupos.items():
    letra_grupo = grupo.split()[-1] 
    
    ganador = tabla.iloc[0]['Equipo']
    segundo = tabla.iloc[1]['Equipo']
    
    clasificados[f'Winner Group {letra_grupo}'] = ganador
    clasificados[f'Runner-up Group {letra_grupo}'] = segundo
    
    info_tercero = tabla.iloc[2]
    info_tercero['Group'] = letra_grupo
    terceros_lugares.append(info_tercero)


df_terceros = pd.DataFrame(terceros_lugares)

df_terceros = df_terceros.sort_values(by='Pts', ascending=False)

mejores_8 = df_terceros.head(8)['Equipo'].tolist()

print(f"Los 8 mejores terceros clasificados son: {mejores_8}")



def obtener_rival(nombre_placeholder):
   
    if nombre_placeholder in clasificados:
        return clasificados[nombre_placeholder]
    

    elif '3rd Group' in nombre_placeholder:
        if len(mejores_8) > 0:
            
            return mejores_8.pop(0) 
        else:
            return "Eliminado" 
            
    
    else:
        return nombre_placeholder


for index, row in df_fixture_ronda_32.iterrows():
    home = row['Local']
    away = row['Visitante']
    
    
    df_fixture_ronda_32.at[index, 'Local'] = obtener_rival(home)
    df_fixture_ronda_32.at[index, 'Visitante'] = obtener_rival(away)


df_fixture_ronda_32["Ganador"]="?"
df_fixture_ronda_32

Los 8 mejores terceros clasificados son: ['South Korea', 'Scotland', 'Egypt', 'Norway', 'Ghana', 'Ukraine', 'Canada', 'Australia']


C:\Users\Felipe Gomez\AppData\Local\Temp\ipykernel_4564\2380150965.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_tercero['Group'] = letra_grupo
C:\Users\Felipe Gomez\AppData\Local\Temp\ipykernel_4564\2380150965.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_tercero['Group'] = letra_grupo
C:\Users\Felipe Gomez\AppData\Local\Temp\ipykernel_4564\2380150965.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_te

,Local,Marcador,Visitante,Año,Ganador
72,Mexico,Match 73,Wales,2026,?
73,Brazil,Match 76,Japan,2026,?
74,Germany,Match 74,South Korea,2026,?
75,Netherlands,Match 75,Morocco,2026,?
76,Ecuador,Match 78,Senegal,2026,?
77,France,Match 77,Scotland,2026,?
78,South Africa,Match 79,Egypt,2026,?
79,England,Match 80,Norway,2026,?
80,Belgium,Match 82,Ghana,2026,?
81,Paraguay,Match 81,Ukraine,2026,?


In [329]:
def ganador(tabla_actualizada):
    for index, row in tabla_actualizada.iterrows():
        local, visitante=row["Local"], row["Visitante"]
        puntos_local, puntos_visitante = predecir_puntos(local, visitante)
        if puntos_local>puntos_visitante:
            ganador=local
        else:
            ganador=visitante
        tabla_actualizada.loc[index, "Ganador"]=ganador    
    return(tabla_actualizada)    

In [330]:
ganador(df_fixture_ronda_32)

,Local,Marcador,Visitante,Año,Ganador
72,Mexico,Match 73,Wales,2026,Mexico
73,Brazil,Match 76,Japan,2026,Brazil
74,Germany,Match 74,South Korea,2026,Germany
75,Netherlands,Match 75,Morocco,2026,Netherlands
76,Ecuador,Match 78,Senegal,2026,Ecuador
77,France,Match 77,Scotland,2026,France
78,South Africa,Match 79,Egypt,2026,South Africa
79,England,Match 80,Norway,2026,England
80,Belgium,Match 82,Ghana,2026,Belgium
81,Paraguay,Match 81,Ukraine,2026,Paraguay


In [331]:
df_fixture_ronda_16

,Local,Marcador,Visitante,Año
88,Winner Match 73,Match 90,Winner Match 75,2026
89,Winner Match 74,Match 89,Winner Match 77,2026
90,Winner Match 76,Match 91,Winner Match 78,2026
91,Winner Match 79,Match 92,Winner Match 80,2026
92,Winner Match 83,Match 93,Winner Match 84,2026
93,Winner Match 81,Match 94,Winner Match 82,2026
94,Winner Match 86,Match 95,Winner Match 88,2026
95,Winner Match 85,Match 96,Winner Match 87,2026


In [332]:
def actualizar_tabla(df_ronda_1,df_ronda_2):
    for index, row in df_ronda_1.iterrows():
        ganador=df_ronda_1.loc[index,"Ganador"]
        marcador=df_ronda_1.loc[index,"Marcador"]
        df_ronda_2.replace({f"Winner {marcador}":ganador},inplace=True)
    df_ronda_2["Ganador"]="?"     
    return df_ronda_2

In [333]:
actualizar_tabla(df_fixture_ronda_32,df_fixture_ronda_16)

,Local,Marcador,Visitante,Año,Ganador
88,Mexico,Match 90,Netherlands,2026,?
89,Germany,Match 89,France,2026,?
90,Brazil,Match 91,Ecuador,2026,?
91,South Africa,Match 92,England,2026,?
92,Croatia,Match 93,Spain,2026,?
93,Paraguay,Match 94,Belgium,2026,?
94,Argentina,Match 95,United States,2026,?
95,Switzerland,Match 96,Portugal,2026,?


In [334]:
ganador(df_fixture_ronda_16)

,Local,Marcador,Visitante,Año,Ganador
88,Mexico,Match 90,Netherlands,2026,Netherlands
89,Germany,Match 89,France,2026,Germany
90,Brazil,Match 91,Ecuador,2026,Brazil
91,South Africa,Match 92,England,2026,England
92,Croatia,Match 93,Spain,2026,Spain
93,Paraguay,Match 94,Belgium,2026,Belgium
94,Argentina,Match 95,United States,2026,Argentina
95,Switzerland,Match 96,Portugal,2026,Portugal


In [335]:
actualizar_tabla(df_fixture_ronda_16,df_fixture_cuartos)

,Local,Marcador,Visitante,Año,Ganador
96,Germany,Match 97,Netherlands,2026,?
97,Spain,Match 98,Belgium,2026,?
98,Brazil,Match 99,England,2026,?
99,Argentina,Match 100,Portugal,2026,?


In [336]:
df_fixture_cuartos

,Local,Marcador,Visitante,Año,Ganador
96,Germany,Match 97,Netherlands,2026,?
97,Spain,Match 98,Belgium,2026,?
98,Brazil,Match 99,England,2026,?
99,Argentina,Match 100,Portugal,2026,?


In [337]:
ganador(df_fixture_cuartos)

,Local,Marcador,Visitante,Año,Ganador
96,Germany,Match 97,Netherlands,2026,Germany
97,Spain,Match 98,Belgium,2026,Spain
98,Brazil,Match 99,England,2026,Brazil
99,Argentina,Match 100,Portugal,2026,Argentina


In [338]:
actualizar_tabla(df_fixture_cuartos,df_fixture_semi)

,Local,Marcador,Visitante,Año,Ganador
100,Germany,Match 101,Spain,2026,?
101,Brazil,Match 102,Argentina,2026,?


In [339]:
df_fixture_semi

,Local,Marcador,Visitante,Año,Ganador
100,Germany,Match 101,Spain,2026,?
101,Brazil,Match 102,Argentina,2026,?


In [340]:
ganador(df_fixture_semi)

,Local,Marcador,Visitante,Año,Ganador
100,Germany,Match 101,Spain,2026,Germany
101,Brazil,Match 102,Argentina,2026,Brazil


In [341]:
actualizar_tabla(df_fixture_semi,df_fixture_final)

,Local,Marcador,Visitante,Año,Ganador
102,Loser Match 101,Match 103,Loser Match 102,2026,?
103,Germany,Match 104,Brazil,2026,?


In [342]:
ganador(df_fixture_final)

,Local,Marcador,Visitante,Año,Ganador
102,Loser Match 101,Match 103,Loser Match 102,2026,Loser Match 102
103,Germany,Match 104,Brazil,2026,Brazil
